# I. Part 1

## 1. Getting a dataset

In [ ]:
%gui wx
import sys
import os

#####################
# Import of utils.py functions
#####################
# Required to get utils.py and access its functions
notebook_dir = os.path.abspath("")
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(parent_dir)
sys.path.append('.')
from utils import loadFSL, FSLeyesServer, mkdir_no_exist, interactive_MCQ

####################
# DIPY_HOME should be set prior to import of dipy to make sure all downloads point to the right folder
####################
os.environ["DIPY_HOME"] = "/home/jovyan/Data"


#############################
# Loading fsl and freesurfer within Neurodesk
# You can find the list of available other modules by clicking on the "Softwares" tab on the left
#############################
import lmod
await lmod.purge(force=True)
await lmod.load('fsl/6.0.7.4')
await lmod.load('freesurfer/7.4.1')
await lmod.list()

####################
# Setup FSL path
####################
loadFSL()

###################
# Load all relevant libraries for the lab
##################
import fsl.wrappers
from fsl.wrappers import fslmaths

import mne_nirs
import nilearn
from nilearn.datasets import fetch_development_fmri

import mne
import mne_nirs
import dipy
from dipy.data import fetch_bundles_2_subjects, read_bundles_2_subjects
import xml.etree.ElementTree as ET
import os.path as op
import nibabel as nib
import glob

!pip install antspyx
import ants

import openneuro
from mne.datasets import sample
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report


# Useful imports to define the direct download function below
import requests
import urllib.request
from tqdm import tqdm


# FSL function wrappers which we will call from python directly
from fsl.wrappers import fast, bet
from fsl.wrappers.misc import fslroi
from fsl.wrappers import flirt

# General purpose imports to handle paths, files etc
import glob
import pandas as pd
import numpy as np
import json

In [2]:
################
# Start FSLeyes (very neat tool to visualize MRI data of all sorts) within Python
################
fsleyesDisplay = FSLeyesServer()
fsleyesDisplay.show()

10:11:55: Debug: Adding duplicate image handler for 'Windows bitmap file'
10:11:55: Debug: Adding duplicate animation handler for '1' type
10:11:55: Debug: Adding duplicate animation handler for '2' type
10:11:55: Debug: Adding duplicate image handler for 'Windows bitmap file'
10:11:55: Debug: Adding duplicate animation handler for '1' type
10:11:55: Debug: Adding duplicate animation handler for '2' type

(ipykernel_launcher.py:12908): Gtk-CRITICAL **: 10:11:55.416: gtk_window_resize: assertion 'height > 0' failed
10:11:56: Debug: ScreenToClient cannot work when toplevel window is not shown
10:11:56: Debug: ScreenToClient cannot work when toplevel window is not shown
10:11:56: Debug: ScreenToClient cannot work when toplevel window is not shown
10:11:56: Debug: ScreenToClient cannot work when toplevel window is not shown
10:11:56: Debug: ScreenToClient cannot work when toplevel window is not shown
10:11:56: Debug: ScreenToClient cannot work when toplevel window is not shown
10:11:56: De

In [3]:
class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)

def download_url(url, output_path):
    with DownloadProgressBar(unit='B', unit_scale=True,
                             miniters=1, desc=url.split('/')[-1]) as t:
        urllib.request.urlretrieve(url, filename=output_path, reporthook=t.update_to)

def direct_file_download_open_neuro(file_list, file_types, dataset_id, dataset_version, save_dirs):
    # https://openneuro.org/crn/datasets/ds004226/snapshots/1.0.0/files/sub-001:sub-001_scans.tsv
    for i, n in enumerate(file_list):
        subject = n.split('_')[0]
        download_link = 'https://openneuro.org/crn/datasets/{}/snapshots/{}/files/{}:{}:{}'.format(dataset_id, dataset_version, subject, file_types[i],n)
        print('Attempting download from ', download_link)
        download_url(download_link, op.join(save_dirs[i], n))
        print('Ok')
        
def get_json_from_file(fname):
    f = open(fname)
    data = json.load(f)
    f.close()
    return data

In [4]:
# Variables
dataset_id = 'ds000171'
bids_root = '/home/jovyan/Data/dataset/ds000171'
subject = 'control01'  
subject_dir = 'sub-{}'.format(subject)

# Check if the dataset directory exists
if op.exists(bids_root):
    print(f"Removing existing dataset at {bids_root}")
    subprocess.run(["rm", "-rf", bids_root], check=True)

# Download the dataset again
subprocess.run(["openneuro-py", "download", "--dataset", dataset_id,
                "--target-dir", bids_root,
                "--include", op.join(subject_dir, '*'),
               ], check=True)

###################
# Create folders relevant for preprocessing.
###################
mkdir_no_exist(op.join(bids_root, 'derivatives'))
preproc_root = op.join(bids_root, 'derivatives','preprocessed_data')
mkdir_no_exist(preproc_root)
mkdir_no_exist(op.join(preproc_root, 'sub-{}'.format(subject)))
mkdir_no_exist(op.join(preproc_root, 'sub-{}'.format(subject), 'anat'))
mkdir_no_exist(op.join(preproc_root, 'sub-{}'.format(subject), 'func'))
mkdir_no_exist(op.join(preproc_root, 'sub-{}'.format(subject), 'fmap'))

Removing existing dataset at /home/jovyan/Data/dataset/ds000171

👋 Hello! This is openneuro-py 2024.2.0. Great to see you! 🤗

   👉 Please report problems 🤯 and bugs 🪲 at
      https://github.com/hoechenberger/openneuro-py/issues

🌍 Preparing to download ds000171 …


📁 Traversing directories for ds000171 : 436 entities [00:30, 14.07 entities/s]


📥 Retrieving up to 15 files (5 concurrent downloads). 


sub-control01_T1w.nii.gz:   0%|          | 0.00/8.77M [00:00<?, ?B/s]
participants.tsv:   0%|          | 0.00/1.63k [00:00<?, ?B/s]
                                                             
CHANGES:   0%|          | 0.00/45.0 [00:00<?, ?B/s]
                                                   
README:   0%|          | 0.00/0.98k [00:00<?, ?B/s]
                                                   
dataset_description.json:   0%|          | 0.00/2.79k [00:00<?, ?B/s]
sub-control01_T1w.nii.gz:   1%|          | 49.5k/8.77M [00:00<00:32, 279kB/s]
sub-control01_task-music_run-1_events.tsv: 0.00B [00:00, ?B/s]
sub-control01_T1w.nii.gz:   1%|          | 100k/8.77M [00:00<00:32, 283kB/s] 
sub-control01_task-music_run-1_bold.nii.gz:   0%|          | 0.00/33.8M [00:00<?, ?B/s]

sub-control01_task-music_run-3_bold.nii.gz:   0%|          | 0.00/33.9M [00:00<?, ?B/s]


sub-control01_T1w.nii.gz:   2%|▏         | 151k/8.77M [00:00<00:31, 285kB/s]00<?, ?B/s]



sub-control01_task-music_run-2_events.t

✅ Finished downloading ds000171.
 
🧠 Please enjoy your brains.
 


In [5]:
print_dir_tree(bids_root, max_depth=4)

|ds000171/
|--- CHANGES
|--- README
|--- dataset_description.json
|--- participants.tsv
|--- derivatives/
|------ preprocessed_data/
|--------- sub-control01/
|------------ anat/
|------------ fmap/
|------------ func/
|--- sub-control01/
|------ anat/
|--------- sub-control01_T1w.nii.gz
|------ func/
|--------- sub-control01_task-music_run-1_bold.nii.gz
|--------- sub-control01_task-music_run-1_events.tsv
|--------- sub-control01_task-music_run-2_bold.nii.gz
|--------- sub-control01_task-music_run-2_events.tsv
|--------- sub-control01_task-music_run-3_bold.nii.gz
|--------- sub-control01_task-music_run-3_events.tsv
|--------- sub-control01_task-nonmusic_run-4_bold.nii.gz
|--------- sub-control01_task-nonmusic_run-4_events.tsv
|--------- sub-control01_task-nonmusic_run-5_bold.nii.gz
|--------- sub-control01_task-nonmusic_run-5_events.tsv


## 2. Anatomical preprocessing

In [6]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(op.join(bids_root, 'sub-control01', 'anat', 'sub-control01_T1w.nii.gz'))


(ipykernel_launcher.py:12908): Gdk-WARNING **: 10:14:00.800: gdkdrawable-x11.c:952 drawable is not a pixmap or window


In [8]:
def get_skull_stripped_anatomical(bids_root, preproc_root, subject_id, robust=False):
    """
    Function to perform skull-stripping (removing the skull around the brain).
    This is a simple wrapper around the brain extraction tool (BET) in FSL's suite
    It assumes data to be in the BIDS format (which we will cover in the following weeks).
    The method also saves the brain mask which was used to extract the brain.

    The brain extraction is conducted only on the T1w of the participant.

    Parameters
    ----------
    bids_root: string
        The root of the BIDS directory
    preproc_root: string
        The root of the preprocessed data, where the result of the brain extraction will be saved.
    subject_id: string
        Subject ID, the subject on which brain extraction should be conducted.
    robust: bool
        Whether to conduct robust center estimation with BET or not. Default is False.
    """
    # We perform here skull stripping (you'll learn more about it next week!).
    # For now all you need to do is that we remove the bones and flesh from the MRI to get the brain!
    subject = 'sub-{}'.format(subject_id)
    anatomical_path = op.join(bids_root, subject, 'anat', 'sub-{}_T1w.nii.gz'.format(subject_id))
    betted_brain_path = op.join(preproc_root, subject, 'anat', 'sub-{}_T1w'.format(subject_id))
    os.system('bet {} {} -m {}'.format(anatomical_path, betted_brain_path, '-R' if robust else ''))
    print("Done with BET.")

resulting_mask_path = op.join(preproc_root, 'sub-control01', 'anat', 'sub-control01_T1w_mask')
get_skull_stripped_anatomical(bids_root, preproc_root, "control01")

Done with BET.


In [9]:
fsleyesDisplay.load(resulting_mask_path)

In [10]:
get_skull_stripped_anatomical(bids_root, preproc_root, "control01", robust=True)

Done with BET.


In [11]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(op.join(bids_root, 'sub-control01', 'anat', 'sub-control01_T1w.nii.gz'))
fsleyesDisplay.load(resulting_mask_path)

In [12]:
##################################
# Fill me with the code to use your mask
# To help you, we provide you with the skeleton of two potential approaches.
# You can fill either of them. Do not forget to test them by visualizing the result!
# For fslmaths, you can either read the documentation, or execute it without argument by running os.system('fslmaths')
##################################

# Option 1: Pythonic approach.
def apply_python_mask_approach(img_path, mask_path, masked_img_path):
    """
    The first approach, Pythonic way. The goal is, given a mask, to apply it to a T1 where the brain is to be extracted.
    
    YOU SHOULD COMPLETE THE METHOD AS IT DOES NOT WORK RIGHT NOW!

    Parameters
    ----------
    img_path: str
        Path to the image on which we would like to apply the mask (in your case, the T1 with the skull still on). Should be a .nii.gz file
    mask_path: str
        Path to the mask you would like to apply to your image. Should be a .nii.gz file, containing only binary values (0 or 1)
    masked_img_path: str
        Path to which the resulting image will be saved.
    """
    import nibabel as nib

    # Load both the T1 and the mask from disk
    img = nib.load(img_path)
    mask = nib.load(mask_path)
    
    # Load the data from both above images as numpy arrays
    img_data = img.get_fdata()
    mask_data = mask.get_fdata()

    #######################
    # Solution 1
    # Create an empty image and select all which falls in the mask (perhaps the most natural way to think about the mask)
    #######################
    # In all positions within the mask, get the image content
    saved_img_data = np.zeros(img_data.shape)
    saved_img_data[mask_data > 0] = img_data[mask_data > 0]

    # Save the image to disk, by creating a new Nifti image and then writing it out
    img_out = nib.Nifti1Image(saved_img_data,img.affine, img.header)
    nib.save(img_out, masked_img_path)

    #######################
    # Solution 2
    # Another approach is to remove from img_data all that is outside the mask.
    #######################
    
    # In all positions OUTSIDE the mask (where it is equal to 0), throw away the image
    img_data[mask_data == 0] = 0
    
    # Save the image to disk, by creating a new Nifti image and then writing it out
    img_out = nib.Nifti1Image(img_data,img.affine, img.header)
    nib.save(img_out, masked_img_path)
    
def apply_fsl_math_approach(img_path, mask_path, masked_img_path):
    ###########################
    # Solution
    # By reading fslmaths documentation, one can see that the -mas option is exactly what we desire.
    ###########################
    os.system('fslmaths {} -mas {} {}'.format(img_path, mask_path, masked_img_path))
    

anatomical_path = op.join(bids_root, 'sub-control01', 'anat', 'sub-control01_T1w.nii.gz') # The original brain
betted_brain_path = op.join(preproc_root, 'sub-control01', 'anat', 'sub-control01_T1w.nii.gz') # The brain without skull is in the derivatives folder
resulting_mask_path = op.join(preproc_root, 'sub-control01', 'anat', 'sub-control01_T1w_mask.nii.gz') # The mask to use

########################
# CHOOSE ONE OF THE TWO TO IMPLEMENT IT AND LAUNCH IT
########################
apply_fsl_math_approach(anatomical_path, resulting_mask_path, betted_brain_path)
apply_python_mask_approach(anatomical_path, resulting_mask_path, betted_brain_path)

In [13]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(betted_brain_path)

In [14]:
anatomical_path = op.join(bids_root, 'sub-control01', 'anat', 'sub-control01_T1w.nii.gz')
bet_path = op.join(preproc_root, 'sub-control01', 'anat', 'sub-control01_T1w')

#########################
# Solution
# By reading above: we must apply FAST to the brain-extracted image. Thus we must use the BET path brain.
##########################
fast_target = bet_path # Replace with either anatomical_path or bet_path (note: you can try both and decide which is more reasonable!)

[os.remove(f) for f in glob.glob(op.join(preproc_root, 'sub-control01', 'anat', '*fast*'))] # Just to clean the directory in between runs of the cell
segmentation_path = op.join(preproc_root, 'sub-control01', 'anat', 'sub-control01_T1w_fast')
fast(imgs=[fast_target], out=segmentation_path, n_classes=3)

{}

In [16]:
print_dir_tree(bids_root, max_depth=5)

|ds000171/
|--- CHANGES
|--- README
|--- dataset_description.json
|--- participants.tsv
|--- derivatives/
|------ preprocessed_data/
|--------- sub-control01/
|------------ anat/
|--------------- sub-control01_T1w.nii.gz
|--------------- sub-control01_T1w_fast_mixeltype.nii.gz
|--------------- sub-control01_T1w_fast_pve_0.nii.gz
|--------------- sub-control01_T1w_fast_pve_1.nii.gz
|--------------- sub-control01_T1w_fast_pve_2.nii.gz
|--------------- sub-control01_T1w_fast_pveseg.nii.gz
|--------------- sub-control01_T1w_fast_seg.nii.gz
|--------------- sub-control01_T1w_mask.nii.gz
|------------ fmap/
|------------ func/
|--- sub-control01/
|------ anat/
|--------- sub-control01_T1w.nii.gz
|------ func/
|--------- sub-control01_task-music_run-1_bold.nii.gz
|--------- sub-control01_task-music_run-1_events.tsv
|--------- sub-control01_task-music_run-2_bold.nii.gz
|--------- sub-control01_task-music_run-2_events.tsv
|--------- sub-control01_task-music_run-3_bold.nii.gz
|--------- sub-cont

In [17]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(bet_path)
fsleyesDisplay.load(glob.glob(op.join(preproc_root, 'sub-control01', 'anat','*pve_0*'))[0])
fsleyesDisplay.load(glob.glob(op.join(preproc_root, 'sub-control01', 'anat','*pve_1*'))[0])
fsleyesDisplay.load(glob.glob(op.join(preproc_root, 'sub-control01', 'anat','*pve_2*'))[0])
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[1]).cmap = 'Red'
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[2]).cmap = 'Green'
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[3]).cmap = 'Blue'

## 3. Coregistration of images

In [18]:
import threading

def launch_freeview(img_list):
    """
    Wrapper around freeview to launch it with several images.
    This wrapper is necessary to launch freeview in a separate thread, ensuring the notebook is free to do something else.

    Parameters
    ----------
    img_list: list of string
        List of images (files) to load. Assumed by default to be volume files.
    """
    args = []
    
    for i in range(len(img_list)):
        args.append("-v")
        args.append(img_list[i])
    # Run the command
    subprocess.run(["freeview"] + args)

imgList = [op.expandvars('$FSLDIR/data/standard/MNI152_T1_1mm.nii.gz'), 
           op.join(preproc_root, "sub-control01", "anat", "sub-control01_T1w.nii.gz:colormap=greyscale")] # Modify here this list to add any image you want, in .nii.gz format
freeview_thread = threading.Thread(target=launch_freeview, args=(imgList,)) # Remark the (imgList,) when passing to args. This is very important to make Thread work properly

# Start the thread
freeview_thread.start()

print("Freeview is running in a separate thread.")

Freeview is running in a separate thread.


QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-jovyan'


In [19]:
from fsl.wrappers import flirt

# The two images
subject_id = 'control01'
subject_anatomical = op.join(preproc_root, 'sub-{}'.format(subject_id), 'anat', 'sub-control01_T1w')
mni_template = op.expandvars(op.join('$FSLDIR', 'data', 'standard', 'MNI152_T1_1mm_brain'))

###################
# Select which image should be target or reference
# ANSWER:
# The subject anatomical will most often be the target, as the template is usually where we want to map our subjects for 
# group comparison.
# There are cases, however, where we may want the subject anatomical to be the reference. This is the case when we want to map an 
# atlas to a subject while preserving the subject as much as possible.
# We showcase here the case where the subject is chosen as target.
##################
target = subject_anatomical
reference = mni_template
result = op.join(preproc_root, 'sub-{}'.format(subject_id), 'anat', 'sub-{}_T1w_mni'.format(subject_id))
flirt(target, reference, out=result)


Final result: 
0.003341 0.005724 -1.116060 204.820462 
-0.933200 0.500882 0.006703 178.661677 
0.501569 1.049029 -0.003770 -149.723790 
0.000000 0.000000 0.000000 1.000000 



{}

In [20]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(reference) 
fsleyesDisplay.load(result)

In [21]:
#######
# Solution
# We consider only within-modalities costs, as the two images belong to the same modality: least squares and normalized correlation ratio.
# Remark correlation ratio also technically works
#######
possible_costs = ['mutualinfo', 'corratio', 'normcorr', 'normmi', 'leastsq', 'labeldiff']
costs_to_consider = [ 'leastsq', 'normcorr' ]

for c in costs_to_consider:
    flirt(target, reference, out=result + '_' + c, cost=c)


Final result: 
3.458371 1.648305 3.627273 -990.534765 
-2.861950 1.235282 4.578044 -54.943603 
0.433791 -5.372595 2.300280 588.875671 
0.000000 0.000000 0.000000 1.000000 


Final result: 
0.003107 0.005315 -1.116440 204.932830 
-0.935071 0.500262 0.007008 178.944986 
0.501467 1.049276 -0.004127 -149.788866 
0.000000 0.000000 0.000000 1.000000 



In [22]:
for c in costs_to_consider:
    fsleyesDisplay.load(result + '_' + c)

In [23]:
moving_image = ants.image_read(target + '.nii.gz')
fixed_image = ants.image_read(reference + '.nii.gz')

# Compute the transformation (non linear) to put align the moving image to the fixed image
transformation = ants.registration(fixed=fixed_image, moving=moving_image, type_of_transform = 'SyN' )

# After the transformation has been computed, apply it
warpedImage = ants.apply_transforms(fixed=fixed_image, moving=moving_image, transformlist=transformation['fwdtransforms'])

# Save the image to disk
resultAnts = op.join(preproc_root, 'sub-{}'.format(subject_id), 'anat', 'sub-{}_T1w_mni_SyN.nii.gz'.format(subject_id))
ants.image_write(warpedImage, resultAnts)

# Inspect the results with FSLeyes or freeview, as you prefer :)

In [24]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(reference)
fsleyesDisplay.load(result)
fsleyesDisplay.load(resultAnts)

In [25]:
os.system('fsl_anat')

Usage: fsl_anat [options] -i <structural image>
       fsl_anat [options] -d <existing anat directory>
 
Arguments (You may specify one or more of):
  -i <strucural image>         filename of input image (for one image only)
  -d <anat dir>                directory name for existing .anat directory where this script will be run in place
  -o <output directory>        basename of directory for output (default is input image basename followed by .anat)
  --clobber                    if .anat directory exist (as specified by -o or default from -i) then delete it and make a new one
  --strongbias                 used for images with very strong bias fields
  --weakbias                   used for images with smoother, more typical, bias fields (default setting)
  --noreorient                 turn off step that does reorientation 2 standard (fslreorient2std)
  --nocrop                     turn off step that does automated cropping (robustfov)
  --nobias                     turn off steps tha

0

In [26]:
import shutil

def fsl_anat_wrapped(anatomical_target, output_path):
    os.system('fsl_anat -i {} --clobber --nosubcortseg -o {}'.format(anatomical_target,output_path))
    # Now move all files from the output_path.anat folder created by FSL to 
    # the actual output_path
    fsl_anat_path = output_path+'.anat'
    files_to_move = glob.glob(op.join(fsl_anat_path, '*'))
    for f in files_to_move:
        shutil.move(f, op.join(output_path, op.split(f)[1]))
    
    # Remove the output_path.anat folder
    os.rmdir(fsl_anat_path)

In [27]:
fsl_anat_wrapped(anatomical_path, op.join(preproc_root, 'sub-control01', 'anat'))

Mon Oct 21 11:13:39 UTC 2024
Reorienting to standard orientation
Mon Oct 21 11:13:45 UTC 2024
Automatically cropping the image
Starting Single Image Segmentation
T1-weighted image
Imagesize : 176 x 256 x 170
Pixelsize : 1.2 x 1 x 1

1 5.15907
2 5.58726
3 5.94017
KMeans Iteration 0
KMeans Iteration 1
KMeans Iteration 2
KMeans Iteration 3
KMeans Iteration 4
KMeans Iteration 5
KMeans Iteration 6
KMeans Iteration 7
KMeans Iteration 8
KMeans Iteration 9
KMeans Iteration 10
KMeans Iteration 11
KMeans Iteration 12
KMeans Iteration 13
KMeans Iteration 14
Tanaka Iteration 0 bias field 10
Tanaka-inner-loop-iteration=0 MRFWeightsTotal=2.0141e+07 beta=0.02
Tanaka-inner-loop-iteration=1 MRFWeightsTotal=2.04326e+07 beta=0.02
Tanaka-inner-loop-iteration=2 MRFWeightsTotal=2.04422e+07 beta=0.02
Tanaka-inner-loop-iteration=3 MRFWeightsTotal=2.04426e+07 beta=0.02
Tanaka-inner-loop-iteration=4 MRFWeightsTotal=2.04426e+07 beta=0.02
 CLASS 1 MEAN 100.155 STDDEV 131.718 CLASS 2 MEAN 257.478 STDDEV 69.432 CLA

In [28]:
print_dir_tree(bids_root, max_depth=5)

|ds000171/
|--- CHANGES
|--- README
|--- dataset_description.json
|--- participants.tsv
|--- derivatives/
|------ preprocessed_data/
|--------- sub-control01/
|------------ anat/
|--------------- MNI152_T1_2mm_brain_mask_dil1.nii.gz
|--------------- MNI_to_T1_nonlin_field.nii.gz
|--------------- T1.nii.gz
|--------------- T12std_skullcon.mat
|--------------- T1_biascorr.nii.gz
|--------------- T1_biascorr_bet_skull.nii.gz
|--------------- T1_biascorr_brain.nii.gz
|--------------- T1_biascorr_brain_mask.nii.gz
|--------------- T1_fast_bias.nii.gz
|--------------- T1_fast_mixeltype.nii.gz
|--------------- T1_fast_pve_0.nii.gz
|--------------- T1_fast_pve_1.nii.gz
|--------------- T1_fast_pve_2.nii.gz
|--------------- T1_fast_pveseg.nii.gz
|--------------- T1_fast_restore.nii.gz
|--------------- T1_fast_seg.nii.gz
|--------------- T1_fullfov.nii.gz
|--------------- T1_nonroi2roi.mat
|--------------- T1_orig.nii.gz
|--------------- T1_orig2roi.mat
|--------------- T1_orig2std.mat
|--------

In [30]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(op.expandvars(op.join('$FSLDIR', 'data', 'standard', 'MNI152_T1_1mm')))
fsleyesDisplay.load(op.join(preproc_root, 'sub-control01', 'anat', 'T1_to_MNI_lin'))
fsleyesDisplay.load(op.join(preproc_root, 'sub-control01', 'anat', 'T1_to_MNI_nonlin'))

## 4. fMRI preprocessing